In [65]:
import yfinance as yf
import pandas as pd
from typing import List
import csv

In [66]:
def load_etfs_from_file(path: str)-> List:
    with open(path, newline='') as csvfile:
        result = []
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for ticker, amount, ccy in reader:
            result.append((ticker, int(amount), ccy))
        return result

In [67]:
def load_stocks_from_file(path: str)-> List:
    with open(path, newline='') as csvfile:
        result = []
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for ticker, amount in reader:
            result.append((ticker, int(amount)))
        return result

In [68]:
GBP_to_USD = yf.Ticker("GBPUSD=X").info['regularMarketPrice']
print(GBP_to_USD)

1.3808912


In [69]:
EUR_to_USD = yf.Ticker("EURUSD=X").info['regularMarketPrice']
print(EUR_to_USD)

1.186521


In [70]:
def get_stock_info(ticker : str)-> dict:
    return yf.Ticker(ticker).info

In [71]:
def get_stock_price(ticker: str)-> dict:
    return yf.Ticker(ticker).history(period="1d").iloc[0,3]

In [72]:
def process_stock_info(info: dict, amount: int) -> dict:
    total_val = amount * info['regularMarketPrice']
    ccy = info['currency']
    if ccy == 'GBp':
        total_val = (total_val * GBP_to_USD) / 100
        
    
    return {"ticker": info['symbol'],
            "price": info['regularMarketPrice'],
            "amount": amount,
            "BV": info['bookValue'],
            "P/B": info['priceToBook'],
            "total_val": total_val,
            "dividend_rate": info['dividendRate'],
            "yield": info['dividendYield'],
            "industry": info.get('industry', ""),
            "currency": ccy}

In [73]:
def process_eur_etf(ticker: str, price: float, amount: int, ccy: str)-> dict:
    total_val = amount * price

    if ccy == 'GBp':
        total_val = (total_val * GBP_to_USD) / 100
    if ccy == 'EUR':
        total_val = total_val * EUR_to_USD
    if ccy == 'GBP':
        total_val = (total_val * GBP_to_USD) 
        
    return {"ticker": ticker,
            "price": price,
            "amount": amount,
            "total_val": total_val,
            "currency": ccy
        
    }

In [74]:
portfolio = load_stocks_from_file('data/stocks2.csv')

In [75]:
eur_etfs = load_etfs_from_file('data/etfs2.csv')
print(eur_etfs)

[('SPY', 100, 'USD')]


In [76]:
etfs = []
for ticker, amount, ccy in eur_etfs:
    print(ticker)
    etfs.append(process_eur_etf(ticker, get_stock_price(ticker), amount, ccy))

SPY


In [77]:
stocks = []
for ticker, amount in portfolio:
    print(ticker)
    stocks.append(process_stock_info(get_stock_info(ticker), amount))
    
df = pd.DataFrame(stocks)    

ACRE


In [78]:
df_etfs = pd.DataFrame(etfs)

In [79]:
merged_portfolio = df.append(df_etfs, sort=False)
total_portfolio_val = merged_portfolio["total_val"].sum()
merged_portfolio['%portfolio'] = (merged_portfolio['total_val'] / total_portfolio_val) * 100

In [80]:
merged_portfolio.sort_values(by='total_val', ascending=False)

,ticker,price,amount,BV,P/B,total_val,dividend_rate,yield,industry,currency,%portfolio
0,SPY,414.920013,100,NaN,NaN,41492.001343,NaN,NaN,NaN,USD,96.634604
0,ACRE,14.450000,100,14.254,1.013751,1445.000000,1.32,0.0785,REIT—Mortgage,USD,3.365396


In [81]:
print(total_portfolio_val)

42937.00134277344
